# Tutorial 3: Handles and Running MD

MiMiCPy is a python wrapper for preparing and executing MiMiC runs. For details on installation and compatibility, please refer to https://github.com/bharurn/mimicpy.

This is the third tutorial on how to get MiMiCPy running, and introduces the concept of handles and dealing with running MD simulations. Please refer to the previous tutorial on setting up the system and topology before reading this.

## Introduction to Handles

Handles are classes that control the execution of Gromacs and CPMD simulations, and work on top of the host. There are four, controlling four main steps required for executing a MiMiC simulation:
- `mimicpy.prepare.MM`: Controls the generation of the Gromacs topology, and simulation box.
- `mimicpy.simulate.MD`: Controls the execution of classical MD simulations using Gromacs.
- `mimicpy.prepare.QM`: Control the creation of the QM region and generation of the CPMD input script.
- `mimicpy.simulate.MiMiC`: Controls the execution of MiMiC simulations (runs both Gromacs and CPMD).

An object of these handles are created each time its functionality needs to be used. Apart from the specific functionality of each handle, all four have the following common features:
- Run Gromacs commands using the `gmx()` function.
- Run CPMD commands using the `cpmd()` function.
- Maintains a log of the standard ouput/error from all the above commands, can be accessed using the `log` parameter. This is usefull for debugging the runs.
- Maintains a `_status` variable, containing information of all the runs so far.

## MD Simulation Handle

### Loading the Handle

MD simulations are controlled by the `simulate.MD` handle.

In [1]:
import mimicpy
md = mimicpy.simulate.MD()

This will start a new `simulate.MD` object with no status information. However, often this is not what we need as we need the handle to have information on the previously generated topology files. For this we can call the `continueFrom()`, passing the `prepare.MM` object if it is available in memory.

In [2]:
md = mimicpy.simulate.MD.continueFrom(prep)

If the `prepare.MM` object is not available in memory (becuase you are running MD after closing the script used to generate the topologies), then the `simulate.MD` object can be loaded from the `_status.yaml` file stored in the working directory.

In [3]:
md = mimicpy.simulate.MD.fromYaml()

Loading status from _status.yaml..


The yaml file is automatically written by every handle each time a new files are generated. This file keeps track of the current topology, tpr, gro, trr, etc. so that a new run can be easily started. The above explained loading procedure is applicable to all four handles.

### MDP Scripts

To run a Gromacs simulation, we need to create an MDP script. MiMiCPy provides an efficient way for dealing with scripts. A script has the following characteristic:
- Each parameter can be set using the dot operator. For example., if mdp is an MDP script object then `mdp.dt = 0.002` will set the dt parameter in the MDP script.
- A parameter can be unset by setting it to `None`. For example., `mdp.tcouple = None` will cause the tcouple option to be ignored.
- Use `_` to represent a `-` in the parameter name. For example, `mdp.cutoff_scheme` will be translated to `cutoff-scheme` in actual script file.
    - Additionally, for CPMD scripts (see next tutorial), a space is represented by a `__`. For example, `annealing__ions` will be translated to `ANNEALING IONS` in the CPMD input file.
- To view the full script, just convert it into a str, by calling `str()` or print it with `print()`.

A new MDP script can be created by calling the class constructor.

In [4]:
mdp = mimicpy.scripts.MDP(name='Test MDP')

Default scripts for energy minimization, NVT, NPT and productionr runs are provided. These can be accessed as shown below.

In [5]:
em = mimicpy.scripts.MDP.defaultEM()
nvt = mimicpy.scripts.MDP.defaultNVT()
npt = mimicpy.scripts.MDP.defaultNPT()
production = mimicpy.scripts.MDP.defaultPRD()
em.name = 'test_script'
em.nsteps = 200
print(em)

; Gromacs MD parameters for test_script generated by MiMiCPy
integrator = steep
emtol = 1000.0
emstep = 0.01
nstlog = 50
nsteps = 200
nstlist = 10
cutoff-scheme = Verlet
ns-type = grid
coulombtype = PME
rcoulomb = 1.0
rvdw = 1.0
pbc = xyz



### Running MD Simulations

Running md simulation are as simple as calling the `run()` function of the `simulate.MD` handle. The MDP script is passed.

In [6]:
md.run(em)

Starting classical MD calculation: test_script..
All files will be saved to the directory test_script..
Gromacs simulation is now running as a background job..


This creates an MDP file, runs Gromacs grompp and then mdrun. All files are saved to a new directory, created with the name of the mdp file `em.name`.
The above command will only run the simulation on the current node. If you want to run it as a Slurm job, a Slurm jobscript must be set. Creating a jobscript is similar to the creating an MDP script.

In [7]:
jbs = mimicpy.scripts.Slurm(shebang='/bin/zsh', name='test_md', time='00:20:00', cpus_per_node=3)
jbs.ntasks = 16
print(jbs)

#!/bin/zsh
#SBATCH --time=00:20:00
#SBATCH --cpus-per-node=3
#SBATCH --ntasks=16
#SBATCH --job-name=test_md
#SBATCH --output=test_md.%J.out




The paramters can be set in the constructor or by the dot operator. This jobscript is then set to the `simulate.MD` object by calling the `setSlurmSettings()` function. Executing `run()` subsequently, will submit the run as a Slurm job.

In [8]:
md.setSlurmSettings(jbs)
md.run(em)

Setting Slurm job settings from jobscript test_md..
Gromacs simulations submmitted as a Slurm job test_md.sh with the job ID 132845..
The host and/or this script can be safely closed..
